**Name: Afia Zahin**    
        project name: Heart-Disease

In [ ]:
!pip install /kaggle/input/ml-models/package/dist/mlmodels-1.0-py2.py3-none-any.whl

In [ ]:
import os,warnings;warnings.filterwarnings("ignore")
import numpy as np;import pandas as pd;import matplotlib.pyplot as plt
import seaborn as sns;sns.set(style='whitegrid',font_scale=0.8)
from mlmodels.gpr_bclassifier import GPRC
%matplotlib inline

In [ ]:
df = pd.read_csv('/kaggle/input/heart-disease-cleveland-uci/heart_cleveland_upload.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
display(df.describe())

In [ ]:
df.condition.value_counts()

In [ ]:
corrMat(df)

In [ ]:
plot2count('age','sex','Age of Patient','Gender of Patient',plt4,[2,1])
lst1 = ['cp','exang','thal','ca']
lst2 = ['Chest Pain Type','Excersised Induced Angina','Thalium Stress Result','Fluorosopy Vessels']
nplot2count(lst1,lst2,plt4,4)

In [ ]:
lst_ecg = ['oldpeak','restecg','slope','condition']
plot1count('oldpeak','oldpeak: ST Depression Relative to Rest',plt4)
plot2count('restecg','slope','restecg: Resting electrocardiography (ECG)','slope: []',plt4,[1,1])

In [ ]:
lst_blood = ['trestbps','thalach','fbs','chol','condition']
plot1count('trestbps','trestbps: Resting Blood Pressure (mmHg)',plt4)
plot1count_ordered('thalach','thalach: Maximum Heart Rate',df['thalach'].value_counts().iloc[:30].index,plt4)
plot2count('fbs','chol','Fasting Blood Sugar','Serum Cholestoral',plt4,[2,10],None,df['chol'].value_counts().iloc[:40].index)

In [ ]:
lst_ohe_feat = ['sex','restecg','slope','fbs','cp','exang','thal','ca']
lst_ohe_out = []
for i in lst_ohe_feat:
    tdf = pd.get_dummies(df[i],i)
    lst_ohe_out.append(tdf)

lst_ohe_out.append(df['condition'])
df_ohe = pd.concat(lst_ohe_out,axis=1) # One Hot Encoding Features df

In [ ]:
''' Draw a Bivariate Seaborn Pairgrid /w KDE density w/ '''
def snsPairGrid(df):

    ''' Plots a Seaborn Pairgrid w/ KDE & scatter plot of df features'''
    g = sns.PairGrid(df,diag_sharey=False,hue='condition',palette=['#2c3e50','#E888BB'])
    g.fig.set_size_inches(13,13)
    g.map_upper(sns.kdeplot,n_levels=5)
    g.map_diag(sns.kdeplot, lw=2)
    g.map_lower(sns.scatterplot,s=30,edgecolor="k",linewidth=1.5,alpha=1)
    g.add_legend()
    plt.tight_layout()

In [ ]:
# We actually have only 5 continuous numerical features, the rest are categorical numbers
numvars_targ = ['age','trestbps','chol','thalach','oldpeak','condition']
snsPairGrid(df[numvars_targ])

In [ ]:
''' Sample Usage for GPRC() '''
from sklearn.datasets import load_iris
from sklearn.dummy import DummyClassifier as DC
iris = load_iris();X = iris.data;y = iris.target
model = DC(strategy="most_frequent");model.fit(X,y);print(f'DC(): {model.score(X,y).round(2)}')
model = GPRC();model.fit(X,y);print(f'GPRC(): {model.score(X,y)}')

In [ ]:
from sklearn.model_selection import GridSearchCV,cross_val_score

lst_theta = [0.01, 0.1, 1, 10, 100, 1000, 5000]
lst_sig = [0.01, 0.1, 1, 10, 100, 1000, 5000]

def modelEval(ldf,lst_theta,lst_sig,feature='condition'):

    # Given a dataframe, split feature/target variable
    X = ldf.copy()
    y = ldf[feature].copy()
    del X[feature]

    # define parameters for gridsearch (theta,sigma)
    param_grid = {'theta': lst_theta,'sigma': lst_sig}

    # split dataset into 5 segments, fit & predict fo each segment
    model = GPRC(opt=False)  # manual hyperparameter model
    model.fit(X,y)

    gscv = GridSearchCV(model,param_grid,cv=5) # 5 fold CV
    gscv.fit(X.values,y.values)
    results = pd.DataFrame(gscv.cv_results_)
    scores = np.array(results.mean_test_score).reshape(7,7)

    # plot the cross validation mean scores of the 5 fold CV
    heatmap1(scores,xlabel='theta',xticklabels=param_grid['theta'],
                    ylabel='sigma',yticklabels=param_grid['sigma'])

ldf1 = df[lst_ecg] # subset of ecg features
modelEval(ldf1,lst_theta,lst_sig)

In [ ]:
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.preprocessing import PolynomialFeatures,StandardScaler

lst_theta = [0.01, 0.1, 1, 10, 100, 1000, 5000]
lst_sig = [0.01, 0.1, 1, 10, 100, 1000, 5000]

# Model Evaluation Function for Polynomial Feature Pipeline
def modelEval2(ldf,lst_theta,lst_sig,feature='condition'):

    # Given a dataframe, split feature/target variable
    y = ldf[feature].copy()
    X = ldf.copy()
    del X[feature]     # remove target variable

    tlst = []
    for i in [2,7]:

        # create a pipeline combining a polynomial feature
        pipe = Pipeline(steps=[('scaler',StandardScaler()),
                               ('poly',PolynomialFeatures(i)),
                               ('model',GPRC(opt=False))])

        # pipepines require slightly different notations w/ __
        param_grid = {'model__theta': lst_theta,'model__sigma': lst_sig}

        gscv2 = GridSearchCV(pipe,param_grid,cv=10)
        gscv2.fit(X,y)
        ypred = gscv2.predict(X)
        results2 = pd.DataFrame(gscv2.cv_results_)
        scores2 = np.array(results2.mean_test_score).reshape(7,7)
        tlst.append(scores2)

    lst_lab = ['theta','sigma'];lst_tit = ['Poly(2)','Poly(7)']
    lst_tick = [param_grid['model__theta'],param_grid['model__sigma']]

    # Plot two Heatmaps side by side for the two Polynomial
    heatmapn(n=2,values=tlst,labels=lst_lab,ticklabels=lst_tick,titles=lst_tit)

modelEval2(ldf1,lst_theta,lst_sig)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

def modelEval3(ldf,hyp,pred_upd,feature='condition'):

    # Given a dataframe, split feature/target variable
    y = ldf[feature].copy()
    X = ldf.copy()
    del X[feature]     # remove target variable

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=73)

    lst_mu = []
    if(pred_upd!=None):
        model = GPRC(theta=hyp[0],sigma=hyp[1],opt=False,mu_in=pred_upd)
    else:
        model = GPRC(theta=hyp[0],sigma=hyp[1],opt=False)

    model.fit(X_train,y_train)
    model.predict(X_train.values);lst_mu.append(model.mu_s)
    model.predict(X_test.values);lst_mu.append(model.mu_s)
    print(f'Training Score: {model.score(X_train.values,y_train.values)}')
    print(f'Test Score: {model.score(X_test.values,y_test.values)}')

    if(pred_upd==None):
        return lst_mu

lst_ldf1 = modelEval3(ldf1,hyp=[10,100],pred_upd=None)

In [ ]:
''' Cross Validation '''
lst_theta = [10,100, 500, 1000, 1500, 2000, 2500]
lst_sig = [0.01,0.1,1.0,10,50,100, 500]

ldf2 = df[lst_blood]
modelEval(ldf2,lst_theta,lst_sig)

In [ ]:
''' Train/Test Split w/  '''
lst_ldf2 = modelEval3(ldf2,hyp=[1000,50],pred_upd=None)

In [ ]:
''' Ensemble Modification Train/Test Split'''
# lst_ldf1 : ECG based model prediction
modelEval3(ldf2,hyp=[1000,50],pred_upd=lst_ldf1)

In [ ]:
lst = ['age','sex']
ldf3 = df[lst+lst_ecg]
modelEval(ldf3,lst_theta,lst_sig)

In [ ]:
''' Numerical Features Model '''
lst_theta = [100, 500, 1000, 1500, 2000, 2500,3000]
lst_sig = [500,1000,1500,2000,2500,3000,3500]

df_num = df[numvars_targ].copy()
modelEval(df_num,lst_theta,lst_sig)

In [ ]:
lst_ldf3 = modelEval3(df_num,hyp=[1500,1500],pred_upd=None)

In [ ]:
''' Categorical Feature Model '''
lst_theta = [10,100, 500, 1000, 1500, 2000, 2500]
lst_sig = [0.01,0.1,1.0,10,50,100, 500]
modelEval(df_ohe,lst_theta,lst_sig)

In [ ]:
''' Ensemble Modification of Categorical Feature Model '''
modelEval3(df_ohe,hyp=[100,50],pred_upd=lst_ldf3)
